In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
import numpy as np

In [9]:
file_path = './assets/diabetes.csv'
df = pd.read_csv(file_path)
df.replace('?', pd.NA, inplace=True)
df.dropna(inplace=True)
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [11]:
correlation_matrix = df.corr()
correlation_matrix

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
Pregnancies,1.000000,0.129459,0.141282,-0.081672,-0.073535,0.017683,-0.033523,0.544341,0.221898
Glucose,0.129459,1.000000,0.152590,0.057328,0.331357,0.221071,0.137337,0.263514,0.466581
BloodPressure,0.141282,0.152590,1.000000,0.207371,0.088933,0.281805,0.041265,0.239528,0.065068
SkinThickness,-0.081672,0.057328,0.207371,1.000000,0.436783,0.392573,0.183928,-0.113970,0.074752
Insulin,-0.073535,0.331357,0.088933,0.436783,1.000000,0.197859,0.185071,-0.042163,0.130548
BMI,0.017683,0.221071,0.281805,0.392573,0.197859,1.000000,0.140647,0.036242,0.292695
DiabetesPedigreeFunction,-0.033523,0.137337,0.041265,0.183928,0.185071,0.140647,1.000000,0.033561,0.173844
Age,0.544341,0.263514,0.239528,-0.113970,-0.042163,0.036242,0.033561,1.000000,0.238356
Outcome,0.221898,0.466581,0.065068,0.074752,0.130548,0.292695,0.173844,0.238356,1.000000


In [22]:
correlation_threshold = 0.5

features_to_remove = set()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > correlation_threshold:
            colname = correlation_matrix.columns[i]
            features_to_remove.add(colname)
print(features_to_remove)

set()


In [23]:
X = df.drop(columns=['Outcome'])
y = df['Outcome']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [35]:
model = Sequential()

model.add(Dense(8, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [36]:
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=2)

y_pred = model.predict(X_test)

y_pred_binary = np.where(y_pred > 0.5, 1, 0)

accuracy = accuracy_score(y_test, y_pred_binary)
print(f'Accuracy on test set: {accuracy}')

Epoch 1/100
17/17 - 1s - loss: 5.3708 - accuracy: 0.6518 - 789ms/epoch - 46ms/step
Epoch 2/100
17/17 - 0s - loss: 5.3708 - accuracy: 0.6518 - 30ms/epoch - 2ms/step
Epoch 3/100
17/17 - 0s - loss: 5.3708 - accuracy: 0.6518 - 31ms/epoch - 2ms/step
Epoch 4/100
17/17 - 0s - loss: 5.3708 - accuracy: 0.6518 - 32ms/epoch - 2ms/step
Epoch 5/100
17/17 - 0s - loss: 5.3708 - accuracy: 0.6518 - 31ms/epoch - 2ms/step
Epoch 6/100
17/17 - 0s - loss: 5.3708 - accuracy: 0.6518 - 30ms/epoch - 2ms/step
Epoch 7/100
17/17 - 0s - loss: 5.3708 - accuracy: 0.6518 - 28ms/epoch - 2ms/step
Epoch 8/100
17/17 - 0s - loss: 5.3708 - accuracy: 0.6518 - 27ms/epoch - 2ms/step
Epoch 9/100
17/17 - 0s - loss: 5.3708 - accuracy: 0.6518 - 28ms/epoch - 2ms/step
Epoch 10/100
17/17 - 0s - loss: 5.3708 - accuracy: 0.6518 - 29ms/epoch - 2ms/step
Epoch 11/100
17/17 - 0s - loss: 5.3708 - accuracy: 0.6518 - 26ms/epoch - 2ms/step
Epoch 12/100
17/17 - 0s - loss: 5.3708 - accuracy: 0.6518 - 28ms/epoch - 2ms/step
Epoch 13/100
17/17 - 0s